### Objetivo do Estudo

##### Avaliar a viabilidade do uso de modelos de linguagem multimodais (LLMs) como GPT-4o para interpretar diagramas de arquitetura de sistemas em nuvem (AWS e Azure), sem a necessidade de treinamento supervisionado ou cadastro manual de ícones, comparando com soluções tradicionais baseadas em visão computacional.

### Metodologia

* Selecionar 10 diagramas reais ou realistas de arquiteturas (5 AWS + 5 GCP).

* Usar GPT-4o para interpretar diretamente a imagem e gerar:

    - Lista de componentes com nomes e funções.

    - Fluxo entre os componentes.

    - Proposta de ameaça STRIDE por componente. (Opcional)

* Avaliar:

    - Precisão da extração

    - Esforço necessário (quantas horas/homens foram necessárias?)

    - Robustez frente a variações visuais

## Critérios de Avaliação

| Critério                         | Descrição                                                                 | Avaliação Esperada         |
|----------------------------------|---------------------------------------------------------------------------|----------------------------|
| **Reconhecimento correto dos ícones** | O modelo identificou corretamente os ícones (ex: EC2, S3, Cloud Run)     | ✅ / ❌                     |
| **Correlação entre componentes** | O modelo entendeu corretamente as relações entre os serviços/componentes  | Boa / Parcial / Ruim       |
| **Fluxo lógico coerente**        | O fluxo de dados faz sentido com a arquitetura representada               | Sim / Parcial / Não        |
| **Sem necessidade de treino**    | O modelo funcionou sem treinamento supervisionado ou ajuste manual        | Sim / Não                  |
| **Suporte a múltiplos estilos**  | O modelo funcionou mesmo com variações visuais, estilos ou fornecedores   | Sim / Parcial / Não        |

---

## Coleta e Preparação das Imagens

1. **Seleção das Imagens**
   - Escolher 10 diagramas no total:
     - `5` diagramas de arquitetura da **AWS**.
     - `5` diagramas de arquitetura da **GCP**.
   - Priorizar diagramas que representem **casos reais**, com:
     - Diferentes estilos visuais (vertical, horizontal, colorido, monocromático).
     - Variações nos ícones (versões antigas e novas).
     - Inclusão de fluxos com setas e anotações textuais.

2. **Critérios de Inclusão**
   - Diagramas públicos ou de documentação oficial.
   - Imagens em formato `.png`, `.jpg`, ou `.pdf` vetorizado (convertido para imagem).
   - Preferência por imagens com **textos legíveis** e boa resolução.

3. **Anotação da Verdade de Base (Ground Truth)**
   - Para cada diagrama, registrar manualmente:
     - Lista de componentes visíveis e suas funções.
     - Conexões entre os componentes.
     - Direção do fluxo de dados (se aplicável).
   - Essa base será usada para comparar os resultados do modelo.

In [ ]:
openai_api_key = ""

In [ ]:
from openai import OpenAI
import base64
import os

# Substitua pela sua chave da OpenAI
client = OpenAI(api_key=openai_api_key)

# Caminho para a imagem de arquitetura
image_path = "dataset/aws_1.png"
image_filename = os.path.basename(image_path)

# Lê e codifica a imagem em base64
with open(image_path, "rb") as f:
    encoded_image = base64.b64encode(f.read()).decode("utf-8")

# Prompt estruturado
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": f"""
Você é um agente especialista em arquitetura de sistemas e irá receber uma imagem de arquitetura para análise geral.

Seus objetivos são:
1. Identificar os componentes visuais presentes na arquitetura. Para cada componente, use a seguinte estrutura: "<Fornecedor> - <Serviço>". Ex: "AWS - EC2", "GCP - Pub/Sub"
2. Explicar brevemente o papel de cada componente na arquitetura, com foco em sua função dentro do fluxo de dados.
3. Descrever o fluxo da aplicação com base nas setas e conexões presentes, indicando origem e destino dos dados entre componentes.

Responda no formato estruturado para facilitar análise posterior. Use listas ou JSON se necessário.
"""
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded_image}"
                },
            }
        ]
    }
]

# Envia para o modelo GPT-4o
stream = client.chat.completions.create(
                model="o4-mini-2025-04-16",
                messages=messages,
)

# Exibe o resultado
print(stream.choices[0].message.content)

Segue a análise estruturada da arquitetura apresentada:

```json
{
  "componentes": [
    { "fornecedor": "AWS", "servico": "AWS WAF" },
    { "fornecedor": "AWS", "servico": "Amazon CloudFront" },
    { "fornecedor": "AWS", "servico": "Amazon API Gateway" },
    { "fornecedor": "AWS", "servico": "Amazon S3 (Site Web)" },
    { "fornecedor": "AWS", "servico": "AWS Lambda (Gerador de Permissões)" },
    { "fornecedor": "AWS", "servico": "AWS Lambda (Validador de Permissões)" },
    { "fornecedor": "AWS", "servico": "Amazon SES" },
    { "fornecedor": "AWS", "servico": "Amazon S3 (Permissões de armazenamento)" },
    { "fornecedor": "AWS", "servico": "Amazon DynamoDB" }
  ],

  "explicacoes": {
    "AWS WAF": "Protege a aplicação de ataques na camada HTTP/S, filtrando requisições antes de chegarem ao CloudFront.",
    "Amazon CloudFront": "CDN que distribui conteúdo estático e faz cache, melhorando latência e escalabilidade do site.",
    "Amazon API Gateway": "Ponto de entrada REST para

: 

In [ ]:
### Usar documentos de clouds próprias para fazer validações alem da OWASP

## Conclusão

Mesmo sem realizar testes diretos com o Azure Computer Vision ou ferramentas similares, a escolha por modelos LLMs como o GPT-4o pode ser justificada por critérios técnicos e estratégicos:

#### 1. Reconhecimento de componentes sem necessidade de treinamento

* O GPT-4o demonstrou, neste experimento, a capacidade de reconhecer corretamente diversos ícones de arquitetura (AWS, GCP) sem qualquer treinamento supervisionado.

* Já o Azure Vision requer treinamento manual com conjuntos de dados rotulados (ex: ícones da AWS) para identificar corretamente cada componente, o que gera alto custo de implementação e manutenção.

#### 2. Suporte a múltiplos estilos e fornecedores

* O GPT-4o é robusto a variações visuais, diferentes estilos de diagramação e iconografia, e consegue interpretar imagens de diferentes fornecedores (AWS, GCP, Azure).

* Ferramentas baseadas em visão computacional tradicional são normalmente especializadas em um estilo visual específico e sofrem quando os ícones mudam (o que ocorre com frequência nas atualizações das Big Techs).

#### 3. Compreensão semântica e de contexto
    
* O uso de LLMs permite interpretar o contexto, função e fluxo de cada componente, algo que soluções de visão tradicionais não conseguem fazer sem integração com NLP ou regras customizadas.

* Além disso, o GPT pode produzir saídas estruturadas, sumarizadas ou em linguagem natural prontas para serem adaptadas para frameworks como STRIDE, algo inviável com APIs puramente visuais.

#### 4. Baixo custo de adaptação e manutenção
    
* Como o GPT funciona zero-shot, não é necessário criar datasets, pipelines de labeling ou retraining sempre que um ícone muda.

* Isso oferece grande economia de tempo e recursos, especialmente em projetos onde há rotatividade de arquiteturas ou fornecedores.